# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Основные шаги:

1. Загрузим данные, путь к файлу: /datasets/autos.csv.
2. Изучим данные. Заполним пропущенные значения и обработаем аномалии в столбцах. Если среди признаков имеются неинформативные, удалим их.
3. Подготовим выборки для обучения моделей.
4. Обучим разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробуем разные гиперпараметры.
5. Проанализируем время обучения, время предсказания и качество моделей.
6. Опираясь на критерии заказчика, выберем лучшую модель, проверим её качество на тестовой выборке.

## 1. Подготовка данных

### 1.1 Импорт необходимых библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')

### 1.2 Загрузка датасета 

In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [5]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### 1.3 Поиск пропусков в данных

In [6]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

В столбцах **VehicleType**, **Gearbox**, **Model**, **FuelType** и **Repaired** имеются пропуски, проанализируем их.

### 1.4 Обработка данных

Для начала определим признаки, влияющие на стоимость.


***VehicleType*** - Тип автомобильного кузова ***влияет*** на стоимость.\
***RegistrationYear*** - Год регистрации ***влияет*** на стоимость, тк показывает сколько времени машина эксплуатируется.\
***Gearbox*** - Тип коробки передач ***влияет*** на стоимость, автомобиль с автоматической коробкой передач дороже.\
***Power*** - Мощность ***влияет*** на стоимость, мощнее двигатель - выше цена.\
***Model*** - Модель ***влияет*** на стоимость.\
***Kilometer*** - Пробег ***влияет*** на стоимость, выше пробег - ниже цена.\
***FuelType*** - Тип топлива указывает на тип двигателя, а тип двигателя ***влияет*** на стоимость.\
***Brand*** - Бренд ***влияет*** на стоимость.\
***Repaired*** - Ремонт машины ***влияет*** на стоимость в зависимости от причины ремонта.


И создадим датафрейм с нужными нам столбцами.

In [7]:
dff = df[['Price',
        'VehicleType',
        'RegistrationYear',
        'Gearbox',
        'Power',
        'Model',
        'Kilometer',
        'FuelType',
        'Brand',
        'Repaired']]

#### Price - целевой признак

In [8]:
# Выведем уникальные значения столбца price
dff['Price'].value_counts()

0        10772
500       5670
1500      5394
1000      4649
1200      4594
         ...  
13180        1
10879        1
2683         1
634          1
8188         1
Name: Price, Length: 3731, dtype: int64

In [9]:
# И удалим их
dff = dff.loc[dff['Price'] != 0]

#### VehicleType

In [10]:
# Заменим пропуски на unknown, тк их невозможно корректно заполнить, а при удалении потеряется слишком много записей
dff['VehicleType'] = dff['VehicleType'].fillna('unknown')

#### RegistrationYear

In [11]:
# Выведем уникальные значения столбца RegistrationYear
sdff = sorted(dff['RegistrationYear'].unique())
print(sdff)

[1000, 1001, 1039, 1111, 1234, 1255, 1300, 1400, 1500, 1600, 1602, 1800, 1910, 1923, 1925, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000, 3200, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555, 5600, 5900, 5911, 6000, 6500, 7000, 7100, 7800, 8000, 8200, 8500, 8888, 9000, 9450, 9999]


In [12]:
# Имеются данные с некорректной датой регистрации, отфильтруем эти записи
dff = dff.loc[(dff['RegistrationYear'] <= 2016) & (dff['RegistrationYear'] >= 1910)]

#### Gearbox

In [13]:
dff['Gearbox'].value_counts()

manual    251160
auto       63003
Name: Gearbox, dtype: int64

In [14]:
# Пропуски с столбце Gearbox заменим на manual, тк этот тип коробки передач встречается чаще.
dff['Gearbox'] = dff['Gearbox'].fillna('manual')

In [15]:
# Изменим тип данных на булевый тип, механика - 1, автомат - 0
dff['Gearbox'] = dff['Gearbox'].replace({'manual': 1, 'auto': 0})

In [16]:
dff['Gearbox'].value_counts()

1    266762
0     63003
Name: Gearbox, dtype: int64

#### Model

In [17]:
# Пропуски в model удаляем, тк их невозможно восстановить или заменить другими значениями
dff = dff.loc[~dff['Model'].isna()]

#### Power

In [18]:
# Получим уникальные значения столбца Power
sdff = sorted(dff['Power'].unique())
print(sdff)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [19]:
# И отфильтруем некорректные значения
dff.loc[(dff['Power'] > 1000) | (dff['Power'] <= 0), 'Power'] = None
dff['Power'] = dff['Power'].fillna(dff.groupby('Model')['Power'].transform('median'))
dff = dff.loc[~dff['Power'].isna()]
dff['Power'] = dff['Power'].astype('int64')

#### Repaired

In [20]:
# Заменим пропущенные значение на yes, отсутствие данных скорее всего подразумевает отсутствие ремонта
dff['Repaired'].fillna("no", inplace=True)

In [21]:
dff['Repaired'].value_counts()

no     283101
yes     31026
Name: Repaired, dtype: int64

In [22]:
# Изменим тип данных на булевый тип, был ремонт - 1, не было ремонта - 0
dff['Repaired'] = dff['Repaired'].replace({'yes': 1, 'no': 0})

In [23]:
dff['Repaired'].value_counts()

0    283101
1     31026
Name: Repaired, dtype: int64

#### FuelType

In [24]:
# Заменим пропуски на среднее по моделям
dff['FuelType'] = dff['FuelType'].fillna(dff.groupby('Model')['FuelType'].transform(lambda x: x.value_counts().idxmax()))

In [25]:
dff.isna().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
Repaired            0
dtype: int64

In [26]:
display(dff)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
0,480,unknown,1993,1,101,golf,150000,petrol,volkswagen,0
2,9800,suv,2004,0,163,grand,125000,gasoline,jeep,0
3,1500,small,2001,1,75,golf,150000,petrol,volkswagen,0
4,3600,small,2008,1,69,fabia,90000,gasoline,skoda,0
5,650,sedan,1995,1,102,3er,150000,petrol,bmw,1
...,...,...,...,...,...,...,...,...,...,...
354362,3200,sedan,2004,1,225,leon,150000,petrol,seat,1
354363,1150,bus,2000,1,125,zafira,150000,petrol,opel,0
354366,1199,convertible,2000,0,101,fortwo,125000,petrol,smart,0
354367,9200,bus,1996,1,102,transporter,150000,gasoline,volkswagen,0


In [27]:
dff['Gearbox'].value_counts() #325 989

1    253888
0     60239
Name: Gearbox, dtype: int64

In [28]:
df['Gearbox'].value_counts() #334 536

manual    268251
auto       66285
Name: Gearbox, dtype: int64

In [29]:
df['Gearbox'].isna().sum()

19833

In [30]:
dff['Repaired'].value_counts()

0    283101
1     31026
Name: Repaired, dtype: int64

### Вывод:
- Импортировал необходимые библиотеки
- Загрузил датасет для работы
- Проверил данные на пропуски
- Выделил необходимые признаки и проверил данные, удалил пропуски и исправил некорректные значения

## 2. Обучение моделей

Обучать модели будем с помощью:
- LinearRegression
- DecisionTreeRegressor
- LGBMRegressor


In [31]:
# Удаляем price
features = dff.drop('Price', axis=1)

In [32]:
#Целевой признак
target = dff['Price']

In [33]:
X_train, X_temp, y_train, y_temp = train_test_split(dff.drop(columns=['Price']), dff['Price'], test_size=0.3, random_state=12345)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

In [34]:
categorical_features = ['VehicleType','FuelType','Brand','Model']
numerical_features = ['RegistrationYear', 'Gearbox', 'Power', 'Kilometer', 'Repaired']
# Кодирование категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

# Преобразуем данные
features = dff.drop(columns=['Price'])  # Удаляем целевой столбец
target = dff['Price']

# Применяем преобразование
ohe_features = preprocessor.fit_transform(features)

# Проверка, что результат преобразования корректен:
print(f"Форма закодированных признаков: {ohe_features.shape}")

Форма закодированных признаков: (314127, 305)


In [35]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

In [36]:
print(f"X_train.shape: {X_train.shape}, X_val.shape: {X_val.shape}")


X_train.shape: (219888, 9), X_val.shape: (47119, 9)


In [37]:
# Кодирование категориальных данных
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

In [38]:
# Линейная регрессия

linear_model = LinearRegression()
linear_model.fit(X_train_transformed, y_train)

LinearRegression()

In [39]:
# Задаём параметры для перебора
ridge_params = {
    'alpha': [0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky']
}

best_params = {}
best_rmse = float("inf")

# Перебираем все комбинации параметров
for alpha in ridge_params['alpha']:
    for solver in ridge_params['solver']:
        ridge_model = Ridge(alpha=alpha, solver=solver)
        ridge_model.fit(X_train_transformed, y_train)
        y_pred = ridge_model.predict(X_val_transformed)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = {'alpha': alpha, 'solver': solver}

print("Лучшие гиперпараметры:", best_params)
print("Лучший результат (RMSE):", best_rmse)

Лучшие гиперпараметры: {'alpha': 1.0, 'solver': 'svd'}
Лучший результат (RMSE): 2653.961787299938


In [40]:
# Преобразуем категориальные признаки в индексы для LightGBM
def encode_categories(df, cat_columns):
    for col in cat_columns:
        df[col] = df[col].astype('category')
    return df

In [41]:
lgbm_X_train_transformed = encode_categories(X_train, categorical_features)
lgbm_X_val_transformed = encode_categories(X_val, categorical_features)


In [42]:
# Задаём параметры для перебора
lgbm_params = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200]
}

best_params = {}
best_rmse = float("inf")

# Перебираем все комбинации параметров
for learning_rate in lgbm_params['learning_rate']:
    for n_estimators in lgbm_params['n_estimators']:
        lgbm_model = lgb.LGBMRegressor(learning_rate=learning_rate, n_estimators=n_estimators)
        lgbm_model.fit(lgbm_X_train_transformed, y_train)
        y_pred = lgbm_model.predict(lgbm_X_val_transformed)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = {'learning_rate': learning_rate, 'n_estimators': n_estimators}

print("Лучшие гиперпараметры:", best_params)
print("Лучший результат (RMSE):", best_rmse)

Лучшие гиперпараметры: {'learning_rate': 0.1, 'n_estimators': 200}
Лучший результат (RMSE): 1618.9087117832075


## 3. Анализ моделей

### 3.1 LinearRegression

RMSE вычисляется на валидационной выборке.
Время обучения = время model.fit(X_train, y_train)
Время предсказания = model.predict(X_valid)

In [43]:
linear_model = LinearRegression()
%time linear_model.fit(X_train_transformed, y_train)

CPU times: user 17.9 s, sys: 13.4 s, total: 31.2 s
Wall time: 31.5 s


LinearRegression()

In [44]:
%time linear_predict = linear_model.predict(X_val_transformed)

CPU times: user 45.3 ms, sys: 58.8 ms, total: 104 ms
Wall time: 97.1 ms


In [45]:
linear_RMSE = mean_squared_error(linear_predict, y_val, squared=False)
print(f"Linear Regression валидационный RMSE: {linear_RMSE}")

Linear Regression валидационный RMSE: 2654.574338848986


### 3.2 Ridge

In [46]:
ridge_model = Ridge(alpha=1.0, solver='svd')
%time ridge_model.fit(X_train_transformed, y_train)

CPU times: user 31.3 s, sys: 22.1 s, total: 53.4 s
Wall time: 53.4 s


Ridge(solver='svd')

In [47]:
%time ridge_predict = ridge_model.predict(X_val_transformed)

CPU times: user 47.4 ms, sys: 34.2 ms, total: 81.6 ms
Wall time: 97.9 ms


In [48]:
ridge_RMSE = mean_squared_error(ridge_predict, y_val, squared=False)
print(f"ridge Regression валидационный RMSE: {ridge_RMSE}")

ridge Regression валидационный RMSE: 2653.961787299938


### 3.3 LGBMRegressor

In [49]:
lgbm_model = LGBMRegressor(
    n_estimators=200, 
    learning_rate=0.1
)
%time lgbm_model.fit(lgbm_X_train_transformed, y_train)

CPU times: user 35.7 s, sys: 0 ns, total: 35.7 s
Wall time: 35.7 s


LGBMRegressor(n_estimators=200)

In [50]:
%time lgbm_predict = lgbm_model.predict(lgbm_X_val_transformed)

CPU times: user 959 ms, sys: 0 ns, total: 959 ms
Wall time: 1 s


In [51]:
lgbm_RMSE = mean_squared_error(lgbm_predict, y_val, squared=False)
print(f"LGBM Regressor валидационный RMSE: {lgbm_RMSE}")

LGBM Regressor валидационный RMSE: 1618.9087117832075


### Вывод: 
В итоге мы получили следующие данные (total time): 
1. LinearRegression:
    - Время обучения - 44.4 s
    - Время предсказания на валидационной выборке - 109 ms
    - RMSE - 2654.574
2. Redge:
    - Время обучения - 53.4 s
    - Время предсказания на валидационной выборке - 93.6 ms
    - RMSE - 2653.961
3. LGBMRegressor:
    - Время обучения - 2min 14s
    - Время предсказания на валидационной выборке - 822 ms
    - RMSE - 1618.908

In [52]:
# Данные
data = [[22.3, 0.059, 2654.574],
        [47.4, 0.084, 2653.961],
        [37.5, 0.864, 1618.908]]
model = ['Linear Regression', 'Ridge', 'LGBMRegressor']
columns = ['fit_time, s', 'predict_time, s', 'RMSE']

df = pd.DataFrame(data=data, index=model, columns=columns)

# Нормализуем показатели:
df['norm_fit_time'] = 1 - (df['fit_time, s'] - df['fit_time, s'].min()) / (df['fit_time, s'].max() - df['fit_time, s'].min())
df['norm_predict_time'] = 1 - (df['predict_time, s'] - df['predict_time, s'].min()) / (df['predict_time, s'].max() - df['predict_time, s'].min())
df['norm_rmse'] = 1 - (df['RMSE'] - df['RMSE'].min()) / (df['RMSE'].max() - df['RMSE'].min())

# Весовые коэффициенты для метрик
fit_time_weight = 0.3       
predict_time_weight = 0.2    
rmse_weight = 0.5          

# Рассчитываем итоговый рейтинг
df['score'] = (
    fit_time_weight * df['norm_fit_time'] +
    predict_time_weight * df['norm_predict_time'] +
    rmse_weight * df['norm_rmse']
)

# Сортируем по рейтингу
result = df[['score']].sort_values(by='score', ascending=False)

pd.DataFrame(data=df, index=model, columns=['fit_time, s', 'predict_time, s', 'RMSE', 'score'])

,"fit_time, s","predict_time, s",RMSE,score
Linear Regression,22.3,0.059,2654.574,0.500000
Ridge,47.4,0.084,2653.961,0.194085
LGBMRegressor,37.5,0.864,1618.908,0.618327


### Ещё вывод:
1. Лучшей моделью по результату рассчетов оказалась модель **LGBMRegressor**, имея итоговый рейтинг **0.618**
2. Второе место заняла модель **Linear Regressionr** - **0.5**
3. Третье место заняла модель **Linear Regression** - **0.194**\
Разница между рейтингом Ridge и LGBM небольшая, но у Ridge слишком большой показатель RMSE, больше 2500, что неудовлетворяет требованиям заказчика, поэтому лучшей считаем LGBM

### Проверка лучшей модели на тестовой выборке

In [53]:

# Преобразование категориальных признаков новых данных
X_test_transformed = encode_categories(X_test, categorical_features)

# Предсказать значения
%time predictions = lgbm_model.predict(X_test_transformed)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse}")

CPU times: user 970 ms, sys: 0 ns, total: 970 ms
Wall time: 967 ms
RMSE: 1583.7887171099358


## ОБЩИЙ ВЫВОД:
В ходе работы я:
- Импортировал необходимые библиотеки
- Загрузил датасет
- Обработал данные
- Обучил модели
- Проанализировал модели на валидационной выборке 
- Нашел лучшую модель
- И проверил лучшую модель на тестовых данных\
Лучшей оказалась модель LGBMRegressor, с рейтингом 0.618, которая на тестовых данных показала результаты:
- RMSE: 1583.7887171099358
- Total time при обучении - total: 37.5 s
- Total time при предсказании - 0.824 s